In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# add venv PATH to shell command PATH
import sys, os
if sys.base_prefix not in os.environ['PATH']:
    os.environ['PATH'] = f"{sys.base_prefix}/bin:{os.environ['PATH']}"

In [2]:
import bentoml
NAME = 'linear_sleep_clipper'

# load data & train model(not used, just placeholder)

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

    
# add parameters for tuning
num_estimators = 100

# train the model
model = RandomForestRegressor(n_estimators=num_estimators)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('predictions: ', predictions)

# log model performance 
mse = mean_squared_error(y_test, predictions)
print("  mse: %f" % mse)

predictions:  [2.   1.   0.   1.   1.65 0.   1.24 1.   0.   1.   2.   1.   0.   2.
 0.   1.88 2.   2.   0.   0.   1.   2.   1.   1.26 1.48 1.88 1.   1.
 2.   2.  ]
  mse: 0.092897


In [4]:
%%writefile {NAME}.py

import bentoml
import time
from bentoml.artifact import SklearnModelArtifact
from bentoml.handlers import ClipperFloatsHandler


@bentoml.env()
@bentoml.artifacts([SklearnModelArtifact('model')])
class BentoSvc(bentoml.BentoService):

    @bentoml.api(ClipperFloatsHandler)
    def predict_clipper(self, inputs):
        a, b = inputs[0]
        x = inputs.shape[0]
        time.sleep(a * x + b)
        return inputs

Overwriting linear_sleep_clipper.py


In [5]:
from linear_sleep_clipper import BentoSvc

bento_svc = BentoSvc()
bento_svc.pack("model", model)
saved_path = bento_svc.save()

[2020-03-24 18:59:26,551] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-03-24 18:59:26,813] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.5.2+144.g2865d83
creating BentoML-0.5.2+144.g2865d83/BentoML.egg-info
creating BentoML-0.5.2+144.g2865d83/bentoml
creating BentoML-0.5.2+144.g2865d83/bentoml/artifact
creating BentoML-0.5.2+144.g2865d83/bentoml/bundler
creating BentoML-0.5.2+144.g2865d83/bentoml/cli
creating BentoML-0.5.2+144.g2865d83/bentoml/clipper
creating BentoML-0.5.2+144.g2865d83/bentoml/configuration
creating BentoML-0.5.2+144.g2865d83/bentoml/configuration/__pycache__
creating BentoML-0.5.2+144.g2865d83/bentoml/deployment
creating BentoML-0.5.2+144.g2865d83/bentoml/deployment/aws_lambda
creating BentoML-0.5.2+144.g2865d83/bentoml/deployment/sagemaker
creating BentoML-0.5.2+144.g2865d83/bentoml/handlers
creating BentoML-0.5.2+144.g2865d83/bentoml/marshal
creating BentoML-0.5.2+144.g2865d83/bentoml/migrations
creating BentoML-0.5.2+144.g2865d83/bentoml/migrations/__pycache__
creating BentoML-0.5.2+144.g2865d83/bentoml/migrations

In [6]:
from bentoml.utils import detect_free_port
PORT = detect_free_port()
print(PORT)

44213


# Build & Run Bento Service in Docker

In [28]:
from clipper_admin import ClipperConnection, DockerContainerManager
from bentoml.clipper import deploy_bentoml
cl = ClipperConnection(DockerContainerManager())
try:
    cl.start_clipper(cache_size=1)
except:
    cl.connect()

APP_NAME = saved_path.split('/')[-1].lower()
cl.register_application(APP_NAME, 'floats', 'default_pred', 10*1000* 1000)

20-03-25:21:15:39 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-03-25:21:15:42 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmpkak84p9i.yml
20-03-25:21:15:42 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running
20-03-25:21:15:42 INFO     [clipper_admin.py:236] [default-cluster] Application 20200324185926_24748c was successfully registered


In [29]:
clipper_model_name, clipper_model_version = deploy_bentoml(
    cl, saved_path, 'predict_clipper',
    build_envs=dict(
        PIP_INDEX_URL="http://192.168.138.2/simple",
        PIP_TRUSTED_HOST="192.168.138.2",
    )
)

[2020-03-25 21:15:52,826] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-03-25 21:15:52,835] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.2,  but loading from BentoML version 0.5.2+144.g2865d83
[2020-03-25 21:15:54,000] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will 

20-03-25:21:15:54 INFO     [docker_container_manager.py:409] [default-cluster] Found 0 replicas for bentosvc-predict-clipper:20200324185926-24748c. Adding 1
20-03-25:21:15:55 INFO     [clipper_admin.py:724] [default-cluster] Successfully registered model bentosvc-predict-clipper:20200324185926-24748c
20-03-25:21:15:55 INFO     [clipper_admin.py:642] [default-cluster] Done deploying model bentosvc-predict-clipper:20200324185926-24748c.


[2020-03-25 21:15:55,166] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-03-25 21:15:55,175] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.2,  but loading from BentoML version 0.5.2+144.g2865d83


In [30]:
cl.link_model_to_app(APP_NAME, clipper_model_name)
addr = cl.get_query_addr()
clipper_url = f"http://{addr}/{APP_NAME}/predict"
clipper_url

20-03-25:21:16:06 INFO     [clipper_admin.py:303] [default-cluster] Model bentosvc-predict-clipper is now linked to application 20200324185926_24748c


'http://localhost:1337/20200324185926_24748c/predict'

# Test with requests

In [20]:
import json
import random
import requests


headers = {"content-type": "application/json"}
data = json.dumps(
       {"input": [0.0, 2.0 + random.random()/ 10000]}
)

json_response = requests.post(clipper_url, data=data, headers=headers)
print(json_response)
print(json_response.text)

<Response [200]>
{"query_id":419779,"output":"[0.        2.0000768]","default":false}


# Benchmark with locust

In [31]:
%%writefile benchmark_{NAME}.py
from locust import HttpLocust, TaskSet, task, constant
from functools import lru_cache

import pandas as pd
import json
import random

import os

A = float(os.environ.get('A', 0))
B = float(os.environ.get('B', 0))
WAIT = float(os.environ.get('WAIT', 1))

print(dict(A=A, B=B, WAIT=WAIT))


@lru_cache(maxsize=1)
def data_producer():

    headers = {"content-type": "application/json"}

    def _gen_data():
        _A = A + random.random() / 100000.0
        _B = B + random.random() / 10000.0
        data = json.dumps(
               {"input": [_A, _B]}  # random offset to avoid cache
        )
        return headers, data

    return _gen_data


class WebsiteTasks(TaskSet):

    @task
    def index(self):
        headers, data = data_producer()()
        self.client.post("", data, headers=headers)

class WebsiteUser(HttpLocust):
    task_set = WebsiteTasks
    wait_time = constant(WAIT)

Overwriting benchmark_linear_sleep_clipper.py


In [32]:
print(f"locust -f benchmark_{NAME}.py -H {clipper_url}")
!locust -f benchmark_{NAME}.py -H {clipper_url}

locust -f benchmark_linear_sleep_clipper.py -H http://localhost:1337/20200324185926_24748c/predict
[2020-03-26 12:58:19,237] beta/INFO/locust.main: Starting web monitor at http://*:8089
[2020-03-26 12:58:19,237] beta/INFO/locust.main: Starting Locust 0.14.4
[2020-03-26 13:01:15,995] beta/INFO/locust.runners: Hatching and swarming 1000 users at the rate 100 users/s (0 users already running)...
[2020-03-26 13:01:29,470] beta/INFO/locust.runners: All locusts hatched: WebsiteUser: 1000 (0 already running)
^C
[2020-03-26 13:02:00,649] beta/ERROR/stderr: KeyboardInterrupt
[2020-03-26 13:02:00,650] beta/ERROR/stderr: 2020-03-26T05:02:00Z
[2020-03-26 13:02:00,650] beta/ERROR/stderr: 
[2020-03-26 13:02:00,650] beta/INFO/locust.main: Shutting down (exit code 0), bye.
[2020-03-26 13:02:00,650] beta/INFO/locust.main: Cleaning up runner...
[2020-03-26 13:02:00,650] beta/INFO/locust.main: Running teardowns...
 Name                                                          # reqs      # fails     Avg 